# 1. Import Dependencies

In [1]:
%pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import opencv
import cv2 

# Import uuid
import uuid

# Import Operating System
import os

# Import time
import time

# 2. Define Images to Collect

In [3]:
labels = ['thumbsup', 'thumbsdown', 'thankyou', 'livelong']
number_imgs = 20

# 3. Setup Folders 

In [4]:
IMAGES_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'collectedimages')

In [5]:
if not os.path.exists(IMAGES_PATH):
    print("create Tensorflow/workspace/images/collectedimages/ folder")
for label in labels:
    path = os.path.join(IMAGES_PATH, label)
    if not os.path.exists(path):
        print(f"create Tensorflow/workspace/images/collectedimages/{label} folder")

# 4. Capture Images

In [16]:
for label in labels:
    print('Collecting images for {}'.format(label))
    time.sleep(6)
    cap = cv2.VideoCapture(0)
    for imgnum in range(number_imgs):
        print('Collecting image {}'.format(imgnum))
        ret, frame = cap.read()
        imgname = os.path.join(IMAGES_PATH,label,label+'.'+'{}.jpg'.format(str(uuid.uuid1())))
        cv2.imwrite(imgname, frame)
        #cv2.imshow('frame', frame)
        time.sleep(0.1) # if this is high it will cause old images to appere

    cap.release()
    cv2.destroyAllWindows()

# 5. Image Labelling

In [6]:
%pip install --upgrade pyqt5 lxml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 MB 20.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 26.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 23.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.7/346.7 kB 34.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
LABELIMG_PATH = os.path.join('Tensorflow', 'labelimg')

In [10]:
if not os.path.exists(LABELIMG_PATH):
    print(f"create {LABELIMG_PATH} folder")

    LABELIMG_GIT_PATH = os.path.join('Tensorflow', '.git')
    if not os.path.exists(LABELIMG_GIT_PATH):
        print(f"create {LABELIMG_GIT_PATH} folder")
        print(f"Use: git clone https://github.com/tzutalin/labelImg {LABELIMG_PATH}")

In [11]:
print(f"Build: (cd {LABELIMG_PATH} && make gt5py3)")
print(f"Start: (cd {LABELIMG_PATH} && python labelImg.py)")

Build: (cd Tensorflow/labelimg && make gt5py3)
Start: (cd Tensorflow/labelimg && python labelImg.py)


# 6. Move them into a Training and Testing Partition

In [16]:
import os
import shutil
import random

In [19]:
IMAGES_W_PATH = os.path.join('Tensorflow', 'workspace', 'images')
train_path = os.path.abspath(os.path.join(IMAGES_W_PATH, 'train'))
test_path = os.path.abspath(os.path.join(IMAGES_W_PATH, 'test'))
print(train_path)
print(test_path)

/home/anau/Dev/object-detection/Tensorflow/workspace/images/collectedimages/thumbsup
/home/anau/Dev/object-detection/Tensorflow/workspace/images/train
/home/anau/Dev/object-detection/Tensorflow/workspace/images/test


In [25]:
def delete_all_files(folder_path):
    # Iterate through all files in the folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        
        # Check if it's a file (not a directory)
        if os.path.isfile(file_path):
            # Delete the file
            os.remove(file_path)
            print(f"Deleted: {file_path}")

delete_all_files(train_path)
delete_all_files(test_path)

Deleted: /home/anau/Dev/object-detection/Tensorflow/workspace/images/train/livelong.d9f9751e-8c80-11ee-8fc2-a029425aef06.jpg
Deleted: /home/anau/Dev/object-detection/Tensorflow/workspace/images/train/livelong.d9f9751e-8c80-11ee-8fc2-a029425aef06.xml
Deleted: /home/anau/Dev/object-detection/Tensorflow/workspace/images/train/livelong.f96712f8-8c80-11ee-8fc2-a029425aef06.jpg
Deleted: /home/anau/Dev/object-detection/Tensorflow/workspace/images/train/livelong.f96712f8-8c80-11ee-8fc2-a029425aef06.xml
Deleted: /home/anau/Dev/object-detection/Tensorflow/workspace/images/train/livelong.f87b848c-8c80-11ee-8fc2-a029425aef06.jpg
Deleted: /home/anau/Dev/object-detection/Tensorflow/workspace/images/train/livelong.f87b848c-8c80-11ee-8fc2-a029425aef06.xml
Deleted: /home/anau/Dev/object-detection/Tensorflow/workspace/images/train/livelong.f902e5f8-8c80-11ee-8fc2-a029425aef06.jpg
Deleted: /home/anau/Dev/object-detection/Tensorflow/workspace/images/train/livelong.f902e5f8-8c80-11ee-8fc2-a029425aef06.xml


In [26]:

def get_file_pairs(directory):
    file_pairs = []
    for filename in os.listdir(directory):
        name, extension = os.path.splitext(filename)
        if extension.lower() == '.jpg':
            xml_file = os.path.join(directory, name + '.xml')
            if os.path.exists(xml_file):
                file_pairs.append((os.path.join(directory, name)))
    return file_pairs

def random_copy(file_pairs, destination_folder_train, destination_folder_test, percentage):
    num_files_to_copy = int(len(file_pairs) * percentage)
    selected_pairs = random.sample(file_pairs, num_files_to_copy)
    not_selected_pairs = [file for file in file_pairs if file not in selected_pairs]

    for file in selected_pairs:
        shutil.copy(file + '.jpg', destination_folder_train)
        shutil.copy(file + '.xml', destination_folder_train)

    for file in not_selected_pairs:
        shutil.copy(file + '.jpg', destination_folder_test)
        shutil.copy(file + '.xml', destination_folder_test)

file_pairs = []
for label in labels:
    path = os.path.abspath(os.path.join(IMAGES_PATH, label))
    file_pairs = file_pairs + get_file_pairs(path)
random_copy(file_pairs, train_path, test_path, 0.2)

# OPTIONAL - 7. Compress them for Colab Training

In [20]:
TRAIN_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'train')
TEST_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'test')
ARCHIVE_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'archive.tar.gz')

In [21]:
!tar -czf {ARCHIVE_PATH} {TRAIN_PATH} {TEST_PATH}